In [7]:
import pandas as pd
from fredapi import Fred

def download_daily_fred_series(api_key, start_date="2023-01-01", end_date=None):
    """
    Downloads selected macroeconomic series from FRED and converts them to daily frequency.

    Parameters:
        api_key (str): Your FRED API key.
        start_date (str): Start date in 'YYYY-MM-DD'. Default is "2000-01-01".
        end_date (str, optional): End date in 'YYYY-MM-DD'. If None, downloads up to the latest available date.
    
    Returns:
        pd.DataFrame: A DataFrame indexed by date at daily frequency with columns for each series.
    """
    fred = Fred(api_key=api_key)
    
    # Define the FRED series to download
    series_dict = {
        "EFFR": "EFFR",          # Effective Federal Funds Rate
        "Headline_PCE": "PCE",   # Headline Personal Consumption Expenditures Price Index
        "Core_PCE": "PCEPILFE",  # Core PCE Price Index (excludes food and energy)
        "3M_Yield": "DGS3MO",    # 3-Month Treasury Constant Maturity Rate
        "6M_Yield": "DGS6MO",    # 6-Month Treasury Constant Maturity Rate
        "1Y_Yield": "DGS1",      # 1-Year Treasury Constant Maturity Rate
        "2Y_Yield": "DGS2",      # 2-Year Treasury Constant Maturity Rate
        "5Y_Yield": "DGS5",       # 5-Year Treasury Constant Maturity Rate
        "10Y_Yield": "DGS10"
    }
    
    # Download each series and store it as a DataFrame
    data_frames = {}
    for label, series_id in series_dict.items():
        try:
            series_data = fred.get_series(series_id, start_date, end_date)
            # Convert to DataFrame and rename column to the label
            df_series = series_data.to_frame(name=label)
            data_frames[label] = df_series
        except Exception as e:
            print(f"Error downloading {series_id}: {e}")
    
    # Merge all series DataFrames on the date index (outer join)
    df = pd.concat(data_frames.values(), axis=1)
    
    # Create a full daily date range covering all dates in the data
    full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    df = df.reindex(full_index)
    
    # Forward-fill missing values to get a complete daily series
    df = df.fillna(method='ffill')
    
    return df


In [8]:
%pip install fredapi

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
API_KEY = "e30921a9f946f68d5a8b438384681fba"  # Replace with your FRED API key
# Download data from 2000-01-01 to the latest available date
df_daily = download_daily_fred_series(API_KEY, start_date="2023-01-01")

C:\Users\conqu\AppData\Local\Temp\ipykernel_61480\2861675528.py:50: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


In [12]:
df_daily.head()

,EFFR,Headline_PCE,Core_PCE,3M_Yield,6M_Yield,1Y_Yield,2Y_Yield,5Y_Yield,10Y_Yield
2023-01-01,NaN,18475.2,117.526,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-02,NaN,18475.2,117.526,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-03,4.33,18475.2,117.526,4.53,4.77,4.72,4.40,3.94,3.79
2023-01-04,4.33,18475.2,117.526,4.55,4.77,4.71,4.36,3.85,3.69
2023-01-05,4.33,18475.2,117.526,4.66,4.81,4.78,4.45,3.90,3.71


In [13]:
df_daily.describe()

,EFFR,Headline_PCE,Core_PCE,3M_Yield,6M_Yield,1Y_Yield,2Y_Yield,5Y_Yield,10Y_Yield
count,809.000000,811.000000,811.000000,809.000000,809.000000,809.000000,809.000000,809.000000,809.000000
mean,5.011384,19431.203822,121.285496,5.139506,5.054944,4.812138,4.449617,4.111805,4.123053
std,0.385410,635.416372,2.070008,0.434228,0.443540,0.448970,0.391784,0.353037,0.374008
min,4.330000,18475.200000,117.526000,4.300000,4.240000,3.880000,3.490000,3.290000,3.300000
25%,4.580000,18850.500000,119.536000,4.730000,4.740000,4.340000,4.170000,3.870000,3.820000
50%,5.330000,19412.700000,121.418000,5.400000,5.230000,4.870000,4.440000,4.150000,4.190000
75%,5.330000,19905.000000,122.926000,5.470000,5.410000,5.180000,4.810000,4.370000,4.410000
max,5.330000,20412.500000,124.334000,5.630000,5.610000,5.490000,5.190000,4.950000,4.980000
